In [1]:
import torch
torch.cuda.is_available()

In [1]:
!pip3 install -q -r requirements.txt


In [2]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.data import random_split

import numpy as np
import lightning as L

When you convert your model from pytorch to lightning, you need to do these step:

1. LightningModule: nnModule -> L.LightningModule
    - training_step(): do for one batch

In [3]:
# image, mask = next(iter(train_loader))
# print(image.shape)
# print(image.size(0))
# print(image.reshape(image.size(0),-1).shape)
# print(mask.shape)

In [7]:

class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class NN(L.LightningModule):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def training_step(self, batch, batch_idx):
        loss, yhat, y = self._shared_step(batch,batch_idx)
        return loss
    
    def validation_step(self, batch, batch_idx):
        loss, yhat, y = self._shared_step(batch,batch_idx)
        return loss
    
    def test_step(self, batch, batch_idx):
        loss, yhat, y = self._shared_step(batch,batch_idx)
        return loss
    
    def _shared_step(self,batch,batch_idx):
        x, y = batch
        #  reshape the batch data from (C,H,W)->(C*H*W)
        x = x.reshape(x.size(0),-1)
        # predict
        yhat = self.forward(x)
        # calculate loss
        loss = self.loss_fn(yhat, y)
        return loss, yhat, y
    def predict_step(self, batch, batch_idx):
        x, y = batch
        x = x.reshape(x.size(0),-1)
        yhat = self.forward(x)
        preds = torch.argmax(yhat, dim=1)
        return preds 
        
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.001)
        

# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 3

# Load Data
entire_dataset = datasets.MNIST(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True)

train_ds, val_ds = random_split(entire_dataset, [50000, 10000])

test_ds = datasets.MNIST(
    root="dataset/", train=False, transform=transforms.ToTensor(), download=True)

train_loader = DataLoader(dataset=train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=train_ds, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_ds, batch_size=batch_size, shuffle=False)

# Initialize network
model = NN(input_size=input_size, num_classes=num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Trainer
trainer = L.Trainer(accelerator="gpu", devices=[0], min_epochs=1, max_epochs=3, precision=16)
trainer.fit(model, train_loader, val_loader)

# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    # We don't need to keep track of gradients here so we wrap it in torch.no_grad()
    with torch.no_grad():
        # Loop through the data
        for x, y in loader:

            # Move data to device
            x = x.to(device=device)
            y = y.to(device=device)

            # Get to correct shape
            x = x.reshape(x.shape[0], -1)

            # Forward pass
            scores = model(x)
            _, predictions = scores.max(1)

            # Check how many we got correct
            num_correct += (predictions == y).sum()

            # Keep track of number of samples
            num_samples += predictions.size(0)
            acc = num_correct / num_samples
    model.train()
    return acc


# Check accuracy on training & test to see how good our model
model.to(device)
print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on validation set: {check_accuracy(val_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

/home/thanglx/miniconda3/envs/torch/lib/python3.10/site-packages/lightning/fabric/connector.py:563: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


RuntimeError: No CUDA GPUs are available

In [3]:
import torch
torch.cuda.is_available()

False

In [2]:
!pip3 install torch torchvision torchaudio